### Concate train and test files.

In [1]:
import pandas as pd, numpy as np
import pickle
from sklearn.model_selection import train_test_split


In [13]:
# Combine all TRAC test and gold data. Whoops...


trac_fb = pd.read_csv('../Data/TRAC2018_full/converted/trac-gold-set/agr_en_fb_gold.csv')
trac_sm = pd.read_csv('../Data/TRAC2018_full/converted/trac-gold-set/agr_en_tw_gold.csv')

frames = [trac_fb, trac_sm]
df_trac_test_combined = pd.concat(frames)


df_trac_test_combined.to_csv('../Data/TRAC2018_full/converted/trac-gold-set/agr_en_combined_gold.csv', index=None)


In [8]:
#pd.read_csv('../Data/gibert_vua_format/trainData.csv', sep='\t')
pd.read_csv('../Data/hate_speech_mlma/MLMA_convert/level1/test.tsv', sep='\t', usecols=['text','label'])

ValueError: Usecols do not match columns, columns expected but not found: ['text', 'label']

In [2]:

def has_duplicated(l):
    return pd.Series(l).duplicated().sum() > 0

print(has_duplicated(['one', 'two', 'one']))
# True
print(has_duplicated(['one', 'two', 'three']))
# False

True
False


In [22]:
def pickle_data(train_file, test_file, type='csv'):
    '''Concat and convert VUA data to pickle files for Deepmoji.

    Args: 
        train_file (str): filepath to a training datafile.
        test_file (str): filepath to a test_file.
    
    Optional:
        type (str): specify data file format (default is .csv)

    '''

    # 1. Load the data

    cols_to_use = ['text', 'label']
    df_train = pd.read_csv(train_file, usecols=cols_to_use) #, sep='\t', usecols=cols_to_use)
    df_test = pd.read_csv(test_file, usecols=cols_to_use)#sep='\t', usecols=cols_to_use)

    #print(len(df_train), len(df_test))

    # Concat'em
    frames = [df_train, df_test]
    df_train_and_test = pd.concat(frames)

    # Rename columns to deepmoji's format.
    df_train_and_test.rename({'text': 'texts', 'label':'info'}, axis=1, inplace=True)
    
    # Modify info column values 'str' -> {'label':label}
    df_train_and_test['info'] = df_train_and_test['info'].apply(lambda x: {'label' : x})
    
    # Add the extra index columns
    extra_index_cols =  ['train_ind', 'test_ind', 'val_ind']
    df_train_and_test = df_train_and_test.join(pd.DataFrame(columns=extra_index_cols)) #.fillna(np.nan)
    

    # 2. Construct dict for pickeling.
    # Get indeces

    # Split original train data into train and validation sets.
    org_train_ind = (range(0, len(df_train)))

    train_val_split = train_test_split(org_train_ind, test_size=0.1)
    train_ind = train_val_split[0]
    val_ind = train_val_split[1]

    

    # Add new train indeces.
    #df_train_and_test['train_ind'] = df_train_and_test['train_ind'].add(train_ind, ignore_index=True) #doesnt work.
    #df_train_and_test['train_ind'] = pd.Series(train_ind) # this used to work..
    df_train_and_test.iloc[len(val_ind):len(org_train_ind) : , 2] = train_ind # start at validation indec, run until test index.

    # Add val indeces
    #df_train_and_test['val_ind'] = pd.Series(val_ind) # this used to work.
    df_train_and_test.iloc[:len(val_ind) : , 4] = val_ind

    # Add test indeces
    #test_ind = range(len(train_ind), len(df_train_and_test)) # this used to work.
    test_ind = range(len(org_train_ind), len(df_train_and_test)) # new appraoch

    #df_train_and_test.loc[len(train_ind), 'test_ind'] = test_ind # doesnt work ❌
    #df_train_and_test.iloc[len(train_ind) : , 3] =  test_ind # start at end of new train index, run to the end.. ✔ 
    df_train_and_test.iloc[len(org_train_ind) : , 3] =  test_ind
    #df_train_and_test.insert(loc=len(train_ind), column='test_ind', value=test_ind, allow_duplicates=False) # doesnt work ❌
    

    # Check duplicates
    #dupes = df_train_and_test.duplicated(subset=['train_ind'])

    # 3. Pickle the dict.


    output = df_train_and_test.to_dict('list')
    
    with open('../Data/picklemania/trac/trac_data.pkl', 'wb') as handle:
       pickle.dump(output , handle)

    # This is just for testing.
    return output




In [23]:
# Function call

# TRAC
test = pickle_data('../Data/TRAC2018_full/converted/english/agr_en_train.csv' , '../Data/TRAC2018_full/converted/trac-test-package/agr_en_combine_test.csv')

In [24]:
#test.keys()
#test['texts'][:10]

#print(' length', len(test['texts']))

# Check if things align. (✔)

#print(test['texts'][-1])
#print(test['info'][-1:])
#print(test['train_ind'][-1:])

print(len(test['train_ind']))

df_test = pd.DataFrame(test)
df_test


14172


,texts,info,train_ind,test_ind,val_ind
0,Well said sonu..you have courage to stand agai...,{'label': 'OAG'},NaN,NaN,2656.0
1,"Most of Private Banks ATM's Like HDFC, ICICI e...",{'label': 'NAG'},NaN,NaN,4706.0
2,"Now question is, Pakistan will adhere to this?",{'label': 'OAG'},NaN,NaN,7.0
3,Pakistan is comprised of fake muslims who does...,{'label': 'OAG'},NaN,NaN,3212.0
4,"??we r against cow slaughter,so of course it w...",{'label': 'NAG'},NaN,NaN,4141.0
...,...,...,...,...,...
14167,#ShutdownJNU Pls dnt take police actn ths tym....,{'label': nan},NaN,14167.0,NaN
14168,@IchbinUjjaini what the hell? Do you even know...,{'label': nan},NaN,14168.0,NaN
14169,"#standwithJNU one of the worst authorities, sa...",{'label': nan},NaN,14169.0,NaN
14170,#ShutDownJNU where r so called secular ppl.. #...,{'label': nan},NaN,14170.0,NaN


In [39]:
# inspect pickle file

with open('./olid_lvl_c_data.pkl', 'rb') as handle:
    data = pickle.load(handle)

print(len(data), type(data), data.keys())

print(len(data['texts']))

print()

data['info']

5 <class 'dict'> dict_keys(['texts', 'info', 'train_ind', 'test_ind', 'val_ind'])
13453



[{'label': nan},
 {'label': 'IND'},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': 'OTH'},
 {'label': nan},
 {'label': 'GRP'},
 {'label': nan},
 {'label': 'IND'},
 {'label': nan},
 {'label': nan},
 {'label': 'IND'},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': 'OTH'},
 {'label': 'IND'},
 {'label': nan},
 {'label': 'GRP'},
 {'label': 'GRP'},
 {'label': nan},
 {'label': 'IND'},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': 'GRP'},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': 'IND'},
 {'label': 'GRP'},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': nan},
 {'label': 'GRP'},
 {'label': nan},
 {'label': nan},
 {'